In [ ]:
import numpy as np

import matplotlib.pyplot as plt

from astropy.io import fits
from astropy import stats
from astropy.modeling.models import Gaussian2D, Lorentz2D, Moffat2D
from astropy.modeling import fitting
import astropy.units as u

from photutils.aperture import CircularAperture

from mmtwfs.wfs import check_wfsdata, wfsfind

In [ ]:
%load_ext autoreload
%autoreload 2
#%matplotlib widget

In [ ]:
bino_file = "/Volumes/Elements_18TB/wfsdat/20241001/wfs_ff_cal_img_2024.10.01T025918.888.fits"
bino_file = "~/MMT/wfsdat/20250203/wfs_ff_cal_img_2025.02.03T114159.989.fits"
bino_file = "~/MMT/wfsdat/20250203/wfs_ff_cal_img_2025.02.03T123505.342.fits"
data = check_wfsdata(bino_file)

In [ ]:
mean, median, stddev = stats.sigma_clipped_stats(data, sigma=3.0, maxiters=None)

In [ ]:
srcs, wfsfind_fig = wfsfind(data, fwhm=6., threshold=4., std=stddev, plot=True)

In [ ]:
apers = CircularAperture(
    list(zip(srcs['xcentroid'], srcs['ycentroid'])),
    r=12.
)

In [ ]:
masks = apers.to_mask(method='subpixel')
sigma = 0.0
snrs = []
spot = np.zeros(masks[0].shape)
for m in masks:
    subim = m.cutout(data)

    # make co-added spot image for use in calculating the seeing
    if subim.shape == spot.shape:
        spot += subim

    signal = subim.sum()
    noise = np.sqrt(stddev**2 * subim.shape[0] * subim.shape[1])
    snr = signal / noise
    snrs.append(snr)

snrs = np.array(snrs)
fits.writeto("spot.fits", spot, overwrite=True)

back = np.mean([spot[:2, :].mean(), spot[-2:, :].mean(), spot[:, :2].mean(), spot[:, -2:].mean()])
print(back)
spot -= back
g2d = Gaussian2D(amplitude=spot.max(), x_mean=spot.shape[1]/2, y_mean=spot.shape[0]/2)
l2d = Lorentz2D(amplitude=spot.max(), x_0=spot.shape[1]/2, y_0=spot.shape[0]/2)
m2d = Moffat2D(amplitude=spot.max(), x_0=spot.shape[1]/2, y_0=spot.shape[0]/2)

model = m2d
gmodel = g2d
fitter = fitting.DogBoxLSQFitter()
y, x = np.mgrid[:spot.shape[0], :spot.shape[1]]
fit = fitter(model, x, y, spot)
gfit = fitter(gmodel, x, y, spot)

gsigma = 0.5 * (gfit.x_stddev.value + gfit.y_stddev.value)
#print(sigma)
print(fit.fwhm, gsigma * stats.gaussian_sigma_to_fwhm)

In [ ]:
fig, ax = plt.subplots(1, 3)
im1 = ax[0].imshow(spot, origin='lower')
im2 = ax[1].imshow(spot - fit(x, y), origin='lower')
im2 = ax[2].imshow(spot - gfit(x, y), origin='lower')
plt.show()

In [ ]:
gfit

In [ ]:
stats.gaussian_fwhm_to_sigma * fit.fwhm

In [ ]:
(gfit.theta * u.rad).to(u.deg)